In [1]:
%load_ext autoreload
%autoreload 2
# add . to module name
import sys
sys.path.append('../src/')

In [14]:
import folium
import random
import pandas as pd

In [10]:
# center in cologne
m = folium.Map(location=[50.938361, 6.959974], zoom_start=13)

    # --lat-min=50.888361 \
    # --lat-max=50.988361 \
    # --lon-min=6.889974 \
    # --lon-max=6.999974 \

bbox = [50.888361, 6.889974, 50.988361, 6.999974]

folium.Rectangle(
	bounds=[[bbox[0], bbox[1]], [bbox[2], bbox[3]]],
	color='red',
	fill=False,
).add_to(m)

n = 100
# generate n random points in bbox

import random

points = [(random.uniform(bbox[0], bbox[2]), random.uniform(bbox[1], bbox[3])) for i in range(n)]

for point in points:
	folium.CircleMarker(
		location=point,
		radius=2,
		color='#000000',
		fill=True,
		fill_color='#000000'
	).add_to(m)


m

In [17]:
points_df = pd.DataFrame(points, columns=['lat', 'lon'])
points_df.to_csv('../data/bicycle_locations.csv', index=False)